In [ ]:
%%capture
!pip install arviz
!pip install pymc3


In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
from pymc3 import math

from scipy import stats

In [ ]:
#read in the files
with open('blanka_inputs/c_no_punish.npy', 'rb') as f:
    c_no_punish = np.load(f)

with open('blanka_inputs/c_punish.npy', 'rb') as f:
    c_punish = np.load(f)

with open('blanka_inputs/c.npy', 'rb') as f:
    c = np.load(f)

with open('blanka_inputs/Ga_no_punish.npy', 'rb') as f:
    Ga_no_punish = np.load(f)

with open('blanka_inputs/Ga_punish.npy', 'rb') as f:
    Ga_punish = np.load(f)

with open('blanka_inputs/Ga.npy', 'rb') as f:
    Ga = np.load(f)

with open('blanka_inputs/Gc_no_punish.npy', 'rb') as f:
    Gc_no_punish = np.load(f)

with open('blanka_inputs/Gc_punish.npy', 'rb') as f:
    Gc_punish = np.load(f)

with open('blanka_inputs/Gc.npy', 'rb') as f:
    Gc = np.load(f)

with open('blanka_inputs/Gini.npy', 'rb') as f:
    Gini = np.load(f)

with open('blanka_inputs/c_choice_index.npy', 'rb') as f:
    c_choice_index = np.load(f)


In [ ]:
groupSize = 4
ntrials = 10
pi = 1.4
ntokens = 20
ngroups = 244
vals = np.arange(0,21,1) #possible values to contribute - from 0 to 20 tokens

In [ ]:
data = []                       
for people in range(0,4):
    for trials in range (0,10):
        for group in range (0,244):
            data.append([people, trials, group, c[people, trials, group,0]])
            
data = pd.DataFrame(data, columns=['people', 'trials', 'group', 'value'])



In [ ]:
Ginni = pd.DataFrame(Gini,  columns=['gini'])

In [ ]:
Ginni['group'] = Ginni.index

In [ ]:
merged = data.merge(Ginni, on='group', how='left')

In [ ]:
idi = pd.Categorical(merged.people).codes
groupi = pd.Categorical(merged.group).codes
triali = pd.Categorical(merged.trials).codes

In [ ]:
#decay model

with pm.Model() as decay_model:
    beta0_c0 = pm.HalfNormal('beta0_c0', sigma =1)

    betaGini_c0 = pm.Normal(' betaGini_c0', mu = 0, sigma = 1)

    
    beta0_gamma = pm.HalfNormal('beta0_gamma', sigma = 1)

    betaGini_gamma = pm.Normal('betaGini_gamma', mu = 0, sigma = 1)


    sigma_c = pm.Gamma('sigma_c', alpha = 1, beta = 1, shape = (4,244))
    c_0 = pm.Deterministic('c_0', beta0_c0 + betaGini_c0 * Gini_unst[groupi] )
    gamma = pm.Deterministic('gamma', beta0_gamma+ (betaGini_gamma* Gini_unst[groupi]))
    mu_c = pm.Deterministic('mu_c',c_0[groupi] * pm.math.exp(-gamma[groupi] * merged.trials))
    ind_c = pm.Normal('ind_c', mu = mu_c[groupi], sigma = sigma_c[idi, groupi], observed = merged.value, shape = (4, 244))
    trace_1 = pm.sample(return_inferencedata = True, chains=1, random_seed = 88)  

In [ ]:
az.summary(trace_1)

In [ ]:
az.summary(trace_1, var_names = [' betaGini_c0', 'betaGini_gamma', 'beta0_c0', 'beta0_gamma', 'sigma_c', 'gamma'])

In [ ]:
az.plot_trace(trace_1, var_names = [' betaGini_c0', 'betaGini_gamma', 'beta0_c0', 'beta0_gamma'])

In [ ]:
with decay_model:
    ppc = pm.sample_posterior_predictive(
        trace_1, random_seed=88)

In [ ]:
az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=decay_model));

In [ ]:
#THE CC MODEL

In [ ]:
#standardise Gini score for analysis - we need to do this because model is more complex and won't converge otherwise
Gini_unst = Gini
Gini = (Gini - np.average(Gini)) / (np.std(Gini))

In [ ]:
import theano.tensor as tsr

def probit_phi(x):
    """ Probit transform assuming 0 mean and 1 sd """
    mu = 0
    sd = 1
    return 0.5 * (1 + tsr.erf((x - mu) / (sd * tsr.sqrt(2))))

In [ ]:
Gini1 = np.array([Gini,Gini,Gini,Gini])

Trials1 = np.array([0,1,2,3,4,5,6,7,8,9])
Trials1 = np.array([Trials1,Trials1,Trials1,Trials1]).T
Trials1 = Trials1[:,:,None]
Trials1.shape

values = np.array([vals,vals,vals,vals]).T
values = values[:,:,None]


In [ ]:
from pymc3.math import switch, lt
Gb0 = c[:,0,:,0].flatten()
Muc0 = np.array([Ga[0,:,0], Ga[0,:,0], Ga[0,:,0], Ga[0,:,0]]).flatten()

with pm.Model() as CC_Gini:
    #beta0_pbeta = 1
    beta0_pbeta = pm.Normal('beta0_pbeta', mu = 0, sigma = 1)
    #betaGini_pbeta = 1
    betaGini_pbeta = pm.Normal('betaGini_pbeta', mu = 0, sigma = 1)
    
    #------------------------- Individual regression priors -------------------------------------------
    mu_pbeta_probit = pm.Deterministic('mu_pbeta_probit', beta0_pbeta + (betaGini_pbeta*Gini1))
    #mu_pbeta_probit = 1
    sigma_pbeta = pm.Uniform('sigma_pbeta', 1, 100, shape = (4, 244)) #concentration parameter for reparameterised beta distribution
    #sigma_pbeta = 1
    # reparameterising beta prior for slope of beliefs/preferences  in CC model                                                                
    mu_pbeta = probit_phi(mu_pbeta_probit) # probit descale - - mean for cond is lower than overall
    # In the paper, the sigma is added; in the code it's multiplied; we are adding
    shape1_pbeta = mu_pbeta+sigma_pbeta
    shape2_pbeta = (1 - mu_pbeta)+sigma_pbeta

    #------------------------- Model level priors ------------------------------------------------------
    #---------------------------------------------------------------------------------------------------                                                                     
    #decay rate in weighting of beliefs about others - prefs dominate over time
    lam = pm.Beta('lam', 1,1, shape = (4,244))                                                                    
    #parameter weighting of beliefs about what others will contribute, relative to observed contribution     
    gamma = pm.Beta('gamma', 1,1, shape = (4,244))                                                                         
    #~ dunif(0,20) #intercept of linear model relating preferred contributions to possible contribution values
    p0 = pm.Uniform('p0', 0,20, shape = (4,244))
    #slope of linear model relating preferred contributions to possible contribution values                                                                     
    pbeta = pm.Beta('pbeta', shape1_pbeta+1,shape2_pbeta+1, shape = (4,244)) # TODO: check for no +1                                                                    
    
    #vector of preferred contributions for each possible value - linear relationship
    #pvals = pm.Deterministic('pvals', p0 + (pbeta * values)) # pvals[i, s, g]
    pvals =  p0 + (pbeta * values)
    
    #assume beliefs about others on first trial is reflected in first contribution. Simplification for model.
    #make a matrix with the shape of 4 , 10, 244
    #Gb0 = c[:,0,:,0].flatten() imma put this outside the model
    #initial weighting of beliefs about others contributions in choice of own contribution, relative to prefs
    Om0 = pm.Beta('omega0', 1,1, shape = (4*244))
    #Muc0 = np.array([Ga[0,:,0], Ga[0,:,0], Ga[0,:,0], Ga[0,:,0]]).flatten() -- putting this outside
    Out0 = pm.Normal('Out0', Muc0,0.1, observed = merged[merged['trials']==0].value)
    
    
    Gb = []
    Om = []
    Muc = []
    Out = []

    Om.append(Om0)
    Gb.append(Gb0)
    Muc.append(Muc0)
    Out.append(Out0)
    
    for i in range(1,10):
        datat = merged[merged['trials']==i]
        sid = pd.Categorical(datat.people).codes
        gid = pd.Categorical(datat.group).codes
        Om.append(pm.Deterministic('omega'+str(i), Om[i-1]*(1-lam[sid,gid])))
        # We chose to ceil Gb to avoid issues with pvals
        Gb.append(pm.Deterministic('Gb'+str(i), Gb[i-1]*gamma[sid,gid]+(1-gamma[sid,gid] * Ga[i-1, gid,0])))
        # Look at what you made me do.
        Muc.append(Om[i]*Gb[i] + (1-Om[i])*pvals[
            switch(lt(Gb[i], 1),0, switch(lt(Gb[i],2),1, switch(lt(Gb[i],3),2, switch(lt(Gb[i],4),3,
            switch(lt(Gb[i],5),4, switch(lt(Gb[i],6),5,switch(lt(Gb[i],7),6,switch(lt(Gb[i],8),7,
            switch(lt(Gb[i],9),8,switch(lt(Gb[i],10),9,switch(lt(Gb[i],11),10,switch(lt(Gb[i],12),11,
            switch(lt(Gb[i],13),12,switch(lt(Gb[i],14),13,switch(lt(Gb[i],15),14,switch(lt(Gb[i],16),15,
            switch(lt(Gb[i],17),16,switch(lt(Gb[i],18),17,switch(lt(Gb[i],19),18,switch(lt(Gb[i],20),19,20)
            ))))))))))))))))))), sid,gid])
        Out.append(pm.Normal('out'+str(i), Muc[i], 0.1, observed = merged[merged['trials']==i].value))
    bad_trace = pm.sample(200, tune=400, return_inferencedata = True, chains=1, random_seed = 978)
    #done = False
    #seed = 0
    #while not done:
    #    try:
    #        bad_trace = pm.sample(200, tune=400, return_inferencedata = True, chains=1, random_seed = seed)
    #        done = True
    #    except:
    #        seed +=1
            
    #print(seed)
        
    

In [ ]:
az.plot_trace(trace_1, var_names = ['beta0_pbeta', 'betaGini_pbeta', 'mu_pbeta_probit', 'sigma_pbeta'])

In [ ]:
ids = np.arange(0,976)

In [ ]:
# we keep getting an error and we don't know how to fix it